In [1]:
import numpy as np

In [2]:
n = 1000
rng = np.random.default_rng()
z = rng.binomial(n=1,p=0.5, size=n)
x = (1/(1 + np.exp(-(0.5*z+rng.normal(size=n)))) >= 0.5).astype(np.int8)
w = (1/(1 + np.exp(-(0.25*z + 0.25*x +rng.normal(size=n)))) >= 0.5).astype(np.int8)
y = (1/(1 + np.exp(-(0.15*z + 0.15*x + 0.15*w +rng.normal(size=n)))) >= 0.5).astype(np.int8)

In [84]:
n = 1000
rng = np.random.default_rng()
z = rng.binomial(n=1,p=0.5, size=n)
x = (1/(1 + np.exp(-(0.5*z+rng.normal(size=n)))) >= 0.5).astype(np.int8)
w = (1/(1 + np.exp(-(0.25*z + 0.25*x +rng.normal(size=n)))) >= 0.5).astype(np.int8)
y = (1/(1 + np.exp(-(2*x +rng.normal(size=n)))) >= 0.5).astype(np.int8)

In [3]:
r = np.zeros(n)
l = np.zeros(n)
for i in range(n):
    
    p_xwz_1 = np.mean(np.all([x==1, w==w[i], z==z[i]], axis=0))
    p_xwz_0 = np.mean(np.all([x==0, w==w[i], z==z[i]], axis=0))
    p_xz = np.mean(np.all([x==0, z==z[i]], axis=0))
    p_ycond_1 = np.mean(y[np.where(np.all([x==1, w==w[i], z==z[i]], axis=0))])
    p_ycond_0 = np.mean(y[np.where(np.all([x==0, w==w[i], z==z[i]], axis=0))])
    p_wcond = np.mean(w[np.where(np.all([x==0, z==z[i]], axis=0))])
    p_zcond = np.mean(z[np.where(np.all([x==0], axis=0))])

    r[i] = ( ( (1/p_xwz_1)*((y[i]==1) - p_ycond_1)*p_wcond*p_zcond 
    - (1/p_xwz_0)*((y[i]==1) - p_ycond_0)*p_wcond*p_zcond ) +
    (1/p_xz)*p_ycond_1*p_zcond - (1/p_xz)*p_ycond_0*p_zcond)

In [120]:
n = 10000
rng = np.random.default_rng()
z = rng.binomial(n=1,p=0.5, size=n)
x = (1/(1 + np.exp(-(0.1*z+rng.normal(size=n)))) >= 0.5).astype(np.int8)
w = (1/(1 + np.exp(-(x + 0.1*z + rng.normal(size=n)))) >= 0.5).astype(np.int8)
y = (1/(1 + np.exp(-(z + rng.normal(size=n)))) >= 0.5).astype(np.int8)
#y = (1/(1 + np.exp(-(2*x +rng.normal(size=n)))) >= 0.5).astype(np.int8)

de = np.zeros(n)
ie = np.zeros(n)
se = np.zeros(n)
for i in range(n):
    
    p_xwz_1 = np.mean(np.all([x==1, w==w[i], z==z[i]], axis=0))
    p_xwz_0 = np.mean(np.all([x==0, w==w[i], z==z[i]], axis=0))
    p_xz_0 = np.mean(np.all([x==0, z==z[i]], axis=0))
    p_xz_1 = np.mean(np.all([x==1, z==z[i]], axis=0))

    p_x1 = np.mean([x==1])
    p_x0 = 1 - p_x1

    p_ycond_1 = np.mean(y[np.where(np.all([x==1, w==w[i], z==z[i]], axis=0))])
    p_ycond_0 = np.mean(y[np.where(np.all([x==0, w==w[i], z==z[i]], axis=0))])

    p_wcond_x1 = np.mean(w[np.where(np.all([x==1, z==z[i]], axis=0))])
    p_wcond_x0 = np.mean(w[np.where(np.all([x==0, z==z[i]], axis=0))])
    p_zcond_x0 = np.mean(z[np.where(np.all([x==0], axis=0))])
    p_zcond_x1 = np.mean(z[np.where(np.all([x==1], axis=0))])
    p_ycond_w1 = np.mean(y[np.where(np.all([x==0, w==1, z==z[i]], axis=0))])
    p_ycond_w0 = np.mean(y[np.where(np.all([x==0, w==0, z==z[i]], axis=0))])

    
 
    de[i] = ( ( (1/p_xwz_1)*((y[i]==1) - p_ycond_1)*p_wcond_x0*p_zcond_x0 
    - (1/p_xwz_0)*((y[i]==1) - p_ycond_0)*p_wcond_x0*p_zcond_x0  ) +
    (1/p_xz)*p_ycond_1*p_zcond_x0 - (1/p_xz)*p_ycond_0*p_zcond_x0) 

    de[i] += (p_ycond_1-p_ycond_0)*p_wcond_x0*p_zcond_x0

    ie[i] = ( (1/p_xwz_0)*((y[i]==1) - p_ycond_0)*p_wcond_x1*p_zcond_x0 - 
    (1/p_xwz_0)*((y[i]==1) - p_ycond_0)*p_wcond_x0*p_zcond_x0 )
    ie[i] += (1/p_xwz_1 - 1/p_xwz_0)*p_ycond_0*p_zcond_x0
    ie[i] += ( ( (p_xz_1 - p_xz_0)/(p_xz_1*p_x0) ) * 
        (p_wcond_x1*p_ycond_w1 + (1-p_wcond_x1)*p_ycond_w0) )

    #ie[i] += p_ycond_0*p_wcond_x1*p_zcond_x0 - p_ycond_0*p_wcond_x0*p_zcond_x0

    se[i] += ( (1/p_xwz_0)*((y[i]==1) - p_ycond_0)*p_wcond_x0*p_zcond_x1 - 
    (1/p_xwz_0)*((y[i]==1) - p_ycond_0)*p_wcond_x0*p_zcond_x0 )
    se[i] += (1/p_xwz_0)*p_ycond_0*(p_zcond_x1 - p_zcond_x0)
    se[i] += ( ( (p_xz_0 - p_xz_1)/(p_xz_0*p_x1) ) * 
        (p_wcond_x0*p_ycond_w1 + (1-p_wcond_x0)*p_ycond_w0) )

    #se[i] = p_ycond_0*p_wcond_x0*p_zcond_x1 - p_ycond_0*p_wcond_x0*p_zcond_x0

print(np.mean(de))
print(np.mean(ie))
print(np.mean(se))

0.009086063690256444
1.0825635118900505
0.04375007767245285


In [112]:
np.var(ie)

0.007249729898140175